Previously we tried to classify the Self Exclusion variable with moderate success.

We then tried to classify the Risk Rating with no success.

In this notebook we will marry both approaches. 

The purpose of this notebook is to use classifiers to classify the risk rating of 8 vs. not 8

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

In [2]:
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
df = pd.read_csv('df_client.csv')

In [6]:
df.head()

,Country_Code,BR Code,Period,Client,risk_rating,Self_exclude_flag,Variable_1_Y0,Variable_1_Y1,Variable_1_Y2,Variable_1_Y3,...,Variable_28_Y2,Variable_28_Y3,Variable_29_Y0,Variable_29_Y1,Variable_29_Y2,Variable_30_Y0,Variable_30_Y1,Variable_30_Y2,Variable_30_Y3,rr_8
0,0,0,2017Q2,0,7,1,581103.4591,612122.5165,589483.6484,608043.5063,...,601762.9316,574251.4130,577170.3096,594024.8975,616177.8226,588163.8327,623659.1015,608794.9055,574860.551,0
1,0,0,2016Q1,0,7,1,608189.3682,581513.6158,609292.1500,NaN,...,605605.1646,NaN,581951.0166,608354.2362,623470.1198,591055.8212,592011.4052,572734.0028,NaN,0
2,0,0,2015Q4,0,7,1,626775.4450,620338.8464,NaN,NaN,...,NaN,NaN,590490.3620,620329.2616,NaN,626221.0887,572241.0321,NaN,NaN,0
3,0,0,2015Q2,0,7,1,613152.4469,595630.8819,NaN,NaN,...,NaN,NaN,580633.8747,576235.2813,NaN,619098.6619,578761.7137,NaN,NaN,0
4,0,1,2019Q1,1,9,0,615840.2415,603501.2067,587601.9393,610071.5454,...,570273.9177,573434.8221,572413.5987,618435.4264,587802.7283,NaN,NaN,NaN,NaN,0


In [5]:
f1 = df['risk_rating'] == 8

df.loc[:,'rr_8'] = 0
df.loc[f1,'rr_8'] = 1

In [7]:
#cols = ['Variable_16_Y0','Variable_17_Y0', 'Variable_22_Y0','Variable_3_Y0', 
cols = ['Variable_16_Y0','Variable_3_Y0', 'rr_8']

In [8]:
df1 = df.loc[:,cols].dropna().reset_index(drop=True)

In [9]:
df1.head()

,Variable_16_Y0,Variable_3_Y0,rr_8
0,626840.4657,592374.2988,0
1,606359.1289,595587.6726,0
2,627671.8775,576546.0494,0
3,580507.2233,575206.3258,0
4,622216.4303,598898.0024,0


In [10]:
pd.value_counts(df1['rr_8'])

0    19019
1     4216
Name: rr_8, dtype: int64

In [13]:
# split into inputs and outputs
X, y = df1.loc[:,cols[:-1]], df1.loc[:,cols[-1]]
print(X.shape, y.shape)

(23235, 2) (23235,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(15567, 2) (7668, 2) (15567,) (7668,)


In [15]:
# fit the model
model = RandomForestClassifier(random_state=1)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [16]:
# make predictions
yhat = model.predict(X_test)

In [17]:
# evaluate predictions
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.789


In [18]:
cm = confusion_matrix(y_test, yhat) 
print ("Confusion Matrix : \n", cm) 

Confusion Matrix : 
 [[5994  253]
 [1362   59]]


We are getting a high accuracy from our model. 

We were able to predict 0s (not 8 risk rating) 5994/6247 times.

We were able to predict 1s (8 risk rating) 59/1421 times. 

In [20]:
unique, counts = np.unique(y_test, return_counts=True)

print("y_test : \n", np.asarray((unique, counts)).T)

y_test : 
 [[   0 6247]
 [   1 1421]]


In [19]:
unique, counts = np.unique(yhat, return_counts=True)

print("yhat : \n", np.asarray((unique, counts)).T)

yhat : 
 [[   0 7356]
 [   1  312]]


In [22]:
filename = 'rr_8_model.sav'
pickle.dump(model, open(filename, 'wb'))

Although the model was not very successful in predicting the 8s, it was successful in predicting what was not 8. 

We save this model so that it can be used for prediction later